In [3]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.io
import scipy
from scipy import signal
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

import pywt
from scipy import stats

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture as GMM
from sklearn import preprocessing

In [4]:
import math

In [6]:
""" Define schema of datasets
"""

DATABASE = {
    'Database 1': ['female_1',
                  'female_2',
                  'female_3',
                  'male_1',
                  'male_2'],
    'Database 2': ['male_day_1',
                  'male_day_2',
                  'male_day_3']
}

COLUMNS = ['cyl_ch1', 
            'cyl_ch2', 
            'hook_ch1', 
            'hook_ch2', 
            'tip_ch1', 
            'tip_ch2', 
            'palm_ch1', 
            'palm_ch2', 
            'spher_ch1', 
            'spher_ch2', 
            'lat_ch1', 
            'lat_ch2']

LABELS = [
    'Spherical',
    'Tip',
    'Palmar',
    'Lateral',
    'Cylindrical',
    'Hook'
]

COL_MAPPINGS = {
            'cyl_ch1': 'Cylindrical', 
            'cyl_ch2': 'Cylindrical', 
            'hook_ch1': 'Hook', 
            'hook_ch2': 'Hook', 
            'tip_ch1': 'Tip', 
            'tip_ch2': 'Tip', 
            'palm_ch1': 'Palmar', 
            'palm_ch2': 'Palmar', 
            'spher_ch1': 'Spherical', 
            'spher_ch2': 'Spherical', 
            'lat_ch1': 'Lateral', 
            'lat_ch2': 'Lateral'
}

In [7]:
""" Preprocess and standardize dataset into a single dataframe table
"""

DB_NAME = 'Database 1'
dfs = []
for fname in DATABASE[DB_NAME]: 
    tmp_data = scipy.io.loadmat(f'./data/{DB_NAME}/{fname}')
    tmp_data = {k:v for k,v in tmp_data.items() if k in COLUMNS}
    for c in COLUMNS:
        tmp_dfx = pd.DataFrame(tmp_data[c])
        tmp_dfx['identifier'] = fname
        tmp_dfx['label'] = COL_MAPPINGS[c]
        dfs.append(pd.DataFrame(tmp_dfx))

dataset = pd.concat(dfs)
print("Dimensions", dataset.shape)
dataset.head()


Dimensions (1800, 3002)


,0,1,2,3,4,5,6,7,8,9,...,2992,2993,2994,2995,2996,2997,2998,2999,identifier,label
0,0.072198,0.276211,0.429221,0.327214,0.123201,0.097700,0.072198,0.021195,-0.004307,0.174205,...,0.276211,-0.259323,0.072198,1.882814,0.480224,-2.528968,0.097700,0.837247,female_1,Cylindrical
1,0.250710,0.301713,0.199706,0.378218,0.021195,-0.080812,-0.106313,0.021195,0.276211,0.072198,...,-1.304890,0.786244,0.786244,0.939254,1.270775,-0.616346,0.454723,0.021195,female_1,Cylindrical
2,0.123201,0.148703,0.148703,0.123201,-0.004307,-0.157317,-0.029808,0.174205,0.199706,0.250710,...,-0.233821,0.403719,0.046696,-0.080812,0.378218,-0.361330,0.505726,0.607732,female_1,Cylindrical
3,0.531228,-0.106313,-0.284825,-0.335828,-0.182818,0.123201,0.301713,0.352716,0.327214,-0.029808,...,0.709739,0.276211,-0.080812,0.123201,0.735241,1.270775,-0.769356,-1.687415,female_1,Cylindrical
4,-0.310326,-0.182818,0.276211,0.480224,0.352716,0.123201,0.123201,0.097700,-0.029808,-0.080812,...,-0.208320,0.429221,0.378218,0.633234,0.811746,0.403719,-0.182818,-0.412333,female_1,Cylindrical


In [8]:
#combine coupled readouts
new_dfs = []
for lab in LABELS:
    new_temp = dataset[dataset['label'] == lab].values
    new_temp_comb = pd.DataFrame(np.concatenate((new_temp[:150,:3000],new_temp[150:,:3002]),axis=1))
    new_dfs.append(new_temp_comb)


new_dataset = pd.concat(new_dfs)
new_dataset.rename(columns={6001:'label', 6000:'identifier'}, inplace=True)
print("Dimension", new_dataset.shape)
new_dataset.head()

Dimension (900, 6002)


,0,1,2,3,4,5,6,7,8,9,...,5992,5993,5994,5995,5996,5997,5998,5999,identifier,label
0,0.505726,0.531228,0.505726,0.021195,0.046696,-0.080812,0.021195,0.021195,0.123201,0.199706,...,0.227693,0.049139,-0.30797,-0.103908,0.100154,0.15117,-0.614063,-0.358985,female_3,Spherical
1,0.454723,0.174205,-0.182818,-0.386831,0.709739,0.633234,-0.029808,-0.20832,0.0977,0.582231,...,-0.333478,0.100154,0.71234,0.610309,0.559294,-0.410001,0.253201,-0.052892,female_3,Spherical
2,1.245273,0.123201,-2.273952,-2.936995,-0.335828,1.50029,1.398283,2.163332,0.403719,-0.131815,...,0.253201,-0.486524,-0.231447,-0.30797,0.100154,0.865386,0.635817,0.15117,female_3,Spherical
3,-0.080812,-0.182818,0.0977,0.123201,0.276211,0.352716,0.25071,0.199706,0.0977,-0.080812,...,0.227693,-0.818125,-0.052892,0.431755,0.457263,0.304216,0.125662,0.304216,female_3,Spherical
4,0.480224,0.021195,0.429221,0.021195,-0.004307,-0.106313,0.199706,0.199706,0.301713,0.276211,...,-0.103908,-1.124218,0.278708,0.94191,0.686832,0.023631,-1.124218,-0.333478,female_3,Spherical


In [9]:
from cgitb import Hook


def abs_val_filter(data):
    """ Apply an absolute value filter to a DataFrame
    """
    return abs(data.copy())

def butterworth_low_pass_filter(data,     
                                frequency=500,         # sampling frequency
                                lp_filter=5,           # cutoff frequency
                                order=4):
    """
    Create a low pass filter to eliminate noise and smooth EMG data 
    
    The data were collected at a sampling rate of 500 Hz, 
    using as a programming kernel the National Instruments (NI) Labview. 
    The signals were band-pass filtered using a Butterworth Band Pass filter 
    with low and high cutoff at 15Hz and 500Hz respectively and a notch filter at 50Hz 
    to eliminate line interference artifacts.
    """
    lp_filter = lp_filter/(frequency/2)

    # Create a lowpass butterworth signal 
    B, A = scipy.signal.butter(order, 
                               lp_filter, 
                               btype='lowpass')


    # Apply the lowpass signal filter to EMG data
    smooth_emg = scipy.signal.filtfilt(B, 
                                       A, 
                                       data)
    return smooth_emg


def holt_smoothing(data,
                  s_level = 0.5,
                  s_slope = 0.1):
    smoothed = Hook(data[0]).fit(smoothing_level=s_level, smoothing_slope=s_slope).fittedvalues[:]
    print(data[0])
    print(smoothed)
    return smoothed

In [10]:
df_features = new_dataset.iloc[:,:6000].copy()
df_labels = new_dataset.iloc[:, 6001]
df_features = abs_val_filter(df_features)

In [11]:
smoothed_emg_df = butterworth_low_pass_filter(df_features.to_numpy(),
                                             frequency=5000,
                                             lp_filter=25)

In [12]:
frame = smoothed_emg_df.shape[1]
print(frame)

6000


In [136]:
def next_power_of_2(x):
    return 1 if x == 0 else 2 ** (x - 1).bit_length()

In [137]:
fs = 2000

In [138]:
def spectrum(signal, fs):
    m = len(signal)
    n = next_power_of_2(m)
    y = np.round(np.fft.fft(signal, n), 5)
    yh = y[0:int(n / 2 - 1)]
    fh = (fs / n) * np.arange(0, n / 2 - 1, 1)
    power = np.round(np.real(yh * np.conj(yh) / n),5)

    return fh, power

In [139]:
def frequency_ratio(frequency, power):
    power_low = power[(frequency >= 30) & (frequency <= 250)]
    power_high = power[(frequency > 250) & (frequency <= 500)]
    ULC =np.sum(power_low)
    UHC = np.sum(power_high)

    return ULC / UHC

In [140]:
L = smoothed_emg_df.shape[0]
C = smoothed_emg_df.shape[1]

In [141]:
i=0
fr=[]
for i in range(L):
    fr.append([])
    frequency, power = spectrum(smoothed_emg_df[i,:], fs)
    fr[i].append((frequency_ratio(frequency, power)))

<ipython-input-139-700209a04b0c>:7: RuntimeWarning: divide by zero encountered in double_scalars
  return ULC / UHC


In [142]:
fr=np.array(fr)

In [143]:
fr.shape

(900, 1)

In [144]:
i=0
mnp=[]
for i in range(L):
    mnp.append([])
    frequency, power = spectrum(smoothed_emg_df[i,:], fs)
    mnp[i].append(np.sum(power) / len(power))  # Mean power

In [145]:
mnp=np.array(mnp)
mnp.shape

(900, 1)

In [146]:
i=0
tot=[]
for i in range(L):
    tot.append([])
    frequency, power = spectrum(smoothed_emg_df[i,:], fs)
    tot[i].append(np.sum(power))  # Total power
    
tot=np.array(tot)
tot.shape

(900, 1)

In [147]:
def mean_freq(frequency, power):
    num = 0
    den = 0
    for i in range(int(len(power) / 2)):
        num += frequency[i] * power[i]
        den += power[i]

    return num / den

In [148]:
i=0
mnf=[]
for i in range(L):
    mnf.append([])
    frequency, power = spectrum(smoothed_emg_df[i,:], fs)
    mnf[i].append(mean_freq(frequency, power))  # Mean frequency
    
mnf=np.array(mnf)
mnf.shape

(900, 1)

In [149]:
def median_freq(frequency, power):
    power_total = np.sum(power) / 2
    temp = 0
    tol = 0.01
    errel = 1
    i = 0

In [150]:
i=0
mdf=[]
for i in range(L):
    mdf.append([])
    frequency, power = spectrum(smoothed_emg_df[i,:], fs)
    mdf[i].append(median_freq(frequency, power))  # Median frequency
    
mdf=np.array(mdf)
mdf.shape

(900, 1)

In [152]:
i=0
pkf=[]
for i in range(L):
    pkf.append([])
    frequency, power = spectrum(smoothed_emg_df[i,:], fs)
    pkf[i].append(frequency[power.argmax()])  # Peak frequency
    
pkf=np.array(pkf)
pkf.shape

(900, 1)

In [153]:
smoothed_emg_df=np.array(smoothed_emg_df.astype(float))

In [154]:
def wavelet_energy(x, mother, nivel):
    coeffs = pywt.wavedecn(x, wavelet=mother, level=nivel)
    arr, _ = pywt.coeffs_to_array(coeffs)
    et = np.sum(arr ** 2)
    ca = coeffs[0]
    ea = 100 * np.sum(ca ** 2) / et
    ed = []

    for k in range(1, len(coeffs)):
        cd = list(coeffs[k].values())
        cd = np.asarray(cd)
        ed.append(100 * np.sum(cd ** 2) / et)

    return ea, ed

In [155]:

"""
    Compute time-frequency features from signal using sliding window method.
    :param signal: numpy array signal.
    :param frame: sliding window size
    :param step: sliding window step size
    :return: h_wavelet: list
    """
L = smoothed_emg_df.shape[0]
i=0
h_wavelet=[]
for i in range(L):
    h_wavelet.append([])

    E_a, E = wavelet_energy(smoothed_emg_df[i,:], 'db2', 4)
    E.insert(0, E_a)
    E = np.asarray(E) / 100
    h_wavelet[i].append(-np.sum(E * np.log2(E)))
h_wavelet=np.array(h_wavelet)
h_wavelet.shape

(900, 1)

In [184]:
frequancy_feature= np.concatenate([fr,mnp,tot,mnf,mdf,pkf,h_wavelet], axis=1)
frequancy_feature.shape

(900, 7)

In [185]:
frequancy_feature=np.array(frequancy_feature.astype(float))

In [186]:
frequancy_feature = np.nan_to_num(frequancy_feature)
frequancy_feature.shape
#np.array(smoothed_emg_df.astype(float))

(900, 7)

In [187]:
X_train, X_test, y_train, y_test = train_test_split(frequancy_feature, 
                                                    df_labels, 
                                                    test_size=0.2, 
                                                    random_state=5)


In [188]:
""" SVM Classification Results
"""

p = make_pipeline(StandardScaler(), SVC())
clf = GridSearchCV(estimator=p, param_grid={'svc__C': np.logspace(-3, 2, 6), 
                                                     'svc__kernel': ['rbf', 'linear'],
                                                     'svc__degree': [1,2],
                                                     'svc__gamma': np.logspace(-3, 2, 6)
                                                    }, 
                                                      cv=5, refit=True, scoring='accuracy', return_train_score=True)
clf.fit(X_train, y_train)

svm_labels = clf.predict(X_test)

(svm_labels == y_test).value_counts()/len(X_test)

c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered i

False    0.522222
True     0.477778
Name: label, dtype: float64

In [190]:
from sklearn.ensemble import RandomForestClassifier

random_model = RandomForestClassifier(n_estimators=50, random_state = 5, n_jobs = -1)
#Fit
random_model.fit(X_train, y_train)

y_pred = random_model.predict(X_test)

#Checking the accuracy
random_model_accuracy = round(random_model.score(X_train, y_train)*100,2)
print(round(random_model_accuracy, 2), '%')

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [191]:
label_dict = {
    'Spherical':1,
    'Tip':2,
    'Palmar':3,
    'Lateral':4,
    'Cylindrical':5,
    'Hook':6}

col = [label_dict[i] for i in df_labels.to_list()]

In [192]:
svm_res = []
log_res = []
nn_res = []
nb_res = []
kmeans_res = []
gmm_res = []
#rf_res = []
le = preprocessing.LabelEncoder()
le.fit(y_train)

for i in range(0,10):
    X_train, X_test, y_train, y_test = train_test_split(frequancy_feature, 
                                                        df_labels, 
                                                        test_size=0.2, 
                                                        random_state=i)

    clf_svc = make_pipeline(StandardScaler(), SVC(gamma='auto', C=10))
    clf_log = make_pipeline(StandardScaler(), LogisticRegression(C=10, max_iter=1000))
    clf_nn = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(200,200,200), max_iter=2000))
    clf_nb = make_pipeline(StandardScaler(), GaussianNB())
    clf_kmeans = make_pipeline(StandardScaler(), KMeans(n_clusters=len(LABELS)))
    clf_gmm = make_pipeline(StandardScaler(), GMM(n_components=len(LABELS)))
    #clf_rf  = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=50, random_state = 5, n_jobs = -1))


    
    clfs = [clf_svc, 
            clf_log, 
            clf_nn, 
            clf_nb, 
            clf_kmeans, 
            clf_gmm]
            #clf_rf]
            
    for c in clfs:
        c.fit(X_train, y_train)

    svm_labels = clf_svc.predict(X_test)
    log_labels = clf_log.predict(X_test)
    nn_labels = clf_nn.predict(X_test)
    nb_labels = clf_nb.predict(X_test)
    kmeans_labels = clf_kmeans.predict(X_test)
    gmm_labels = clf_gmm.predict(X_test)
    #rf_labels = clf_rf.predict(X_test)


    svm_res.append(((svm_labels == y_test).value_counts()/len(X_test))[1])
    log_res.append(((log_labels == y_test).value_counts()/len(X_test))[1])
    nn_res.append(((nn_labels == y_test).value_counts()/len(X_test))[1])
    nb_res.append(((nb_labels == y_test).value_counts()/len(X_test))[1])
    kmeans_res.append(1 - (sum([abs(i[0] - i[1]) for i in zip(sorted(np.bincount(le.transform(y_test))), 
                                       sorted(np.bincount(kmeans_labels)))])/len(y_test)))
    gmm_res.append(1 - (sum([abs(i[0] - i[1]) for i in zip(sorted(np.bincount(le.transform(y_test))), 
                                       sorted(np.bincount(gmm_labels)))])/len(y_test)))
    #rf_res.append(((nn_labels == y_test).value_counts()/len(X_test))[1])


svm_score = np.max(np.max(np.array(svm_res)))
log_score = np.max(np.max(np.array(log_res)))
nn_score = np.max(np.max(np.array(nn_res)))
nb_score = np.max(np.max(np.array(nb_res)))
kmeans_score = np.max(np.max(np.array(kmeans_res)))
gmm_score = np.max(np.max(np.array(gmm_res)))
#rf_score = np.max(np.max(np.array(rf_res)))


c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
c:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered i

In [193]:
print("SVM Accuracy: ", svm_score)
print("LR Accuracy: ", log_score)
print("Neural Network accuracy:", nn_score)
print("Naive Bayes accuracy", nb_score)
print("K-Means accuracy:", kmeans_score)
print("GMM accuracy", gmm_score)
#print("Random Forest accuracy", rf_score)

SVM Accuracy:  0.5111111111111111
LR Accuracy:  0.4666666666666667
Neural Network accuracy: 0.5666666666666667
Naive Bayes accuracy 0.5
K-Means accuracy: 0.2222222222222222
GMM accuracy 0.4555555555555556
